In [ ]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" spacy==3.1.1

In [ ]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" fastai==2.4.1

## Prepraring for inference

### Preparing the data for inference

In [ ]:
import glob
import numpy
from typing import List
from pandas.core.frame import DataFrame
from fastai.text.all import *

In [ ]:
def drop_cols(df: DataFrame, cols: List) -> DataFrame:
    """drop the given list of columns from the dataframe"""
    return df.drop(cols, axis=1)

def concat_dfs(dfs: List) -> DataFrame:
    """concatenate two dataframes"""
    return pd.concat(dfs)

In [ ]:
df_aug = pd.read_csv('../input/common-lit-datset-with-synonym-replacement/aug_df.csv')
df_aug = drop_cols(df_aug, ['Unnamed: 0', 'id',
                           'url_legal', 'license',
                            'standard_error'])
df_cbt_train = pd.read_csv('../input/commonlit-infusing-data-with-same-domain-data/cbt_train.csv')
df_cbt_train = drop_cols(df_cbt_train, ['Unnamed: 0'])

In [ ]:
df_aug.head(1)

In [ ]:
df_cbt_train.head(1)

Getting the excerpts to a list. This makes it easier to make predictions and later to create the submission file.

In [ ]:
df_cbt_train = df_cbt_train[:10000]
excerpts = df_cbt_train.excerpt.to_list()
excerpts = excerpts
len(excerpts)

## Loading the model

In [ ]:
bkwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/back_final_model')
fwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/forward_final_model')

In [ ]:
bkwd_tst_dl = bkwd_model.dls.test_dl(excerpts)
fwd_tst_dl = fwd_model.dls.test_dl(excerpts)

In [ ]:
bkwd_tst_dl.show_batch()

In [ ]:
fwd_tst_dl.show_batch()

In [ ]:
#bkwd_preds = bkwd_model.get_preds(dl=tst_dl)

In [ ]:
#fwd_preds = fwd_model.get_preds(dl=tst_dl)

In [ ]:
#p = np.array(bkwd_preds[0])
#h = np.array(fwd_preds[0])

In [ ]:
#d = np.mean([p, h], axis=0)

# Predictions

In [ ]:
def ensamble(excerpts, bkwd_model, fwd_model):
    bkwd_preds = np.array(bkwd_model.get_preds(dl=bkwd_tst_dl)[0])
    fwd_preds = np.array(fwd_model.get_preds(dl=fwd_tst_dl)[0])
    return np.mean([bkwd_preds, fwd_preds], axis=0)

In [ ]:
predictions = [pred[0] for pred in ensamble(excerpts, bkwd_model, fwd_model)]
#predictions = [float("{0:.2f}".format(pred.item())) for pred in ensamble(bkwd_model, fwd_model, excerpts)]
#predictions = [float("{0:.2f}".format(model.predict(excerpt)[0][0])) for excerpt in excerpts]

In [ ]:
predictions[:2]

# Creating the submission file

Creating a new dataframe

In [ ]:
df_cbt_train = drop_cols(df_cbt_train, ['target'])

In [ ]:
df_cbt_train.head(1)

In [ ]:
df_cbt_train['target'] = predictions

In [ ]:
df_cbt_train.head(1)

In [ ]:
df_pseudo_labels = concat_dfs([df_aug,df_cbt_train])

Saving the dataframe as csv .

In [ ]:
df_pseudo_labels.to_csv('pseudo_labels.csv', index=False)